In [ ]:
import pandas as pd
import folium

import sys
import os

# Get the absolute path to the scripts folder
scripts_path = os.path.abspath("../scripts")  # Move up one level from notebooks/
sys.path.append(scripts_path)  # Add scripts folder to Python's module search path

# Try importing from config_secret.py, fallback to template if unavailable
try:
    from config_secret import DATA_FOLDER  
except ImportError:
    from config_template import DATA_FOLDER  # Example file for GitHub

# Output paths
csv_output_path = os.path.join(DATA_FOLDER, "ebird_locations.csv")
map_output_path = os.path.join(DATA_FOLDER, "ebird_map.html")

# Ensure the data folder exists
os.makedirs(DATA_FOLDER, exist_ok=True)

# Load CSV from the data folder
file_path = os.path.join(DATA_FOLDER, "MyEBirdData.csv")
df = pd.read_csv(file_path)

# Extract unique locations
location_data = df[['Location ID', 'Location', 'Latitude', 'Longitude']].drop_duplicates()

# Create an interactive map
map_center = [location_data['Latitude'].mean(), location_data['Longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=6)

# Add markers for each location
for _, row in location_data.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,  # Adjust the size (smaller than default markers)
        color="red",  # Border color
        fill=True,
        fill_color="red",  # Inner fill color
        fill_opacity=0.8,
        popup=row['Location']  # Display location name
    ).add_to(m)


for _, row in location_data.iterrows():
    visited_dates = df[df["Location ID"] == row["Location ID"]]["Date"].unique()
    visited_text = "<br>".join(visited_dates) if visited_dates.size > 0 else "No recorded visits"

    popup_content = folium.Popup(
        f"<b>{row['Location']}</b><br><b>Visited:</b><br>{visited_text}",
        max_width=800  # Adjust for wider popups
    )

    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.8,
        popup=popup_content
    ).add_to(m)

# Save csv data for Google MyMaps upload
location_data.to_csv(csv_output_path, index=False)  # Update this to only save this data if variable (or prompt?) is true.

# Save the map (if you still want external access)
m.save(map_output_path)
print(f"✅ Files saved to you data folder named '{os.path.basename(os.path.normpath(DATA_FOLDER))}'")


# Display the map inside the notebook
m
